# Analise de Perfil de clientes

Aqui o objetivo é entender mais sobre o padrão comportamente dos nossos clientes através da analise supervisionada.
Para tanto, utilizaremos duas técnicas diferentes: de centroides e de hierarquização, para centoide direcionamos ao  K-means e Clusterização hierarquica.

Seguiremos um passo a passo que consiste em:


1. Remoção de outliers e PCA

2. Numero de K otimo - Elbow

3. Aplicar a clusterização usando K-means

4. Aplicar a clusterização hierarquica

5. Comparar os resultados - Rand Index e coeifciente de silueta

# A.Remoção de outliers e Redução de Dimensionalidade

In [ ]:
def out_data(df):
    for col in df.columns:
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.01,0.99]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

final_df=out_data(join_tables)

In [ ]:
sample_1 = final_df[:10000]

# B. Número ideal de k

As principais metodologias de definição de K são Cotovelo (elbow) e silhueta. Para definir com extatidão qual seguir vamos fazer  um comparação entre elas, com seus prós e contras:
#### Método do Cotovelo (Elbow)
#### Descrição:
O método do cotovelo consiste em calcular a soma das distâncias quadráticas dentro dos clusters (inertia ou WCSS - Within-Cluster Sum of Squares) para diferentes valores de \( K \). O valor ideal de \( K \) é identificado onde há uma diminuição acentuada na curva, formando um "cotovelo".

#### Prós:
- Simplicidade: Fácil de implementar e interpretar visualmente.
- Intuitivo: A curva do cotovelo fornece uma maneira visualmente clara de selecionar \( K \).
#### Contras:
- Subjetivo: A identificação do "cotovelo" pode ser ambígua, especialmente em datasets onde a curva é suave.
- Escalabilidade: Pode ser computacionalmente caro para grandes datasets, pois requer múltiplas execuções do algoritmo de clustering.

A técnica de normalização escolhidade é o minimos máximos

**Devido ao alto processamento computacional seguiremos com o metodo do cotovelo.**

In [ ]:
# Identificar colunas numéricas e categóricas
num_cols = sample_1.drop(['msno', 'safra', 'churn_1','is_churn', 'plan_list_price','payment_plan_days','is_auto_renew','bd',
 'registration_ate_hoje','transaction_date_ate_expire_transactions_m1','transaction_date_ate_expire',
                          'payment_method_id','payment_plan_days',
 ], axis =1).select_dtypes(include=np.number).columns.tolist()

# Identificar colunas numéricas e categóricas
cat_cols = sample_1.drop(['msno', 'safra', 'churn_1','is_churn', 'plan_list_price','payment_plan_days','is_auto_renew','bd',
 'registration_ate_hoje','transaction_date_ate_expire_transactions_m1','transaction_date_ate_expire',
                          'payment_method_id','payment_plan_days',
 ], axis =1).select_dtypes(include=np.object).columns.tolist()

In [ ]:
#Iniciando parametros do kmeans
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

# Normalizar os dados numéricos
scaler = MinMaxScaler()
scaler = scaler.fit(sample_1[num_cols])
sample_1[num_cols] = scaler.transform(sample_1[num_cols])
#print(scaled)


#create list to hold SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(sample_1[num_cols])
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters nobs ")
plt.ylabel("SSE")
plt.show()

# C. Redução de dimensionalidade (PCA)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pca = PCA(2)
data_pca = pca.fit_transform(sample_1[num_cols])

In [ ]:
plt.scatter(data_pca[:, 0], data_pca[:, 1],
            c=sample_1.is_churn, edgecolor='none', alpha=0.5)
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

# D. Clusterização
Para o agrupamento utilizamos duas técnicas: k-means e GMM

In [ ]:
# Aplicando K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(data_pca)
kmeans_silhouette = silhouette_score(data_pca, kmeans_labels)

# salvando na base
sample_1['kmeans_cluster'] = kmeans_labels


# Métrica de qualidade
print(f'Coeficiente de Silhueta - KMeans: {kmeans_silhouette:.4f}')

In [ ]:
# for inverse transformation
sample_1[num_cols] = scaler.inverse_transform(sample_1[num_cols])
sample_1.groupby('kmeans_cluster').describe()

### Clusterização usando GMM

Para garantir que os dados estejam mais próximos de uma distribuição normal, você pode aplicar uma transformação logarítmica ou a transformação Box-Cox (que é mais flexível) antes da normalização. A seguir, o código atualizado com a transformação Box-Cox (caso os dados sejam estritamente positivos) antes da normalização.

### Explicações:
1. **PowerTransformer (Box-Cox)**: Essa transformação aproxima a normalidade para variáveis positivas, ajustando distribuições assimétricas ou com caudas longas. Use `method='box-cox'` para a transformação Box-Cox.
  - Nota: O Box-Cox requer que os dados sejam **estritamente positivos**.
  - Caso você tenha variáveis que incluem valores zero ou negativos, pode usar `method='yeo-johnson'`, que também é uma transformação de potência, mas não tem essa restrição.
2. **StandardScaler**: Normaliza as variáveis numéricas transformadas para média 0 e desvio padrão 1.
3. **OneHotEncoder**: Codifica as variáveis categóricas em formato binário (dummy variables).
### Resultados:
Esse pipeline agora trata adequadamente os dados numéricos, aplicando uma transformação que aproxima uma distribuição normal e em seguida normaliza os dados para que o **GMM** funcione melhor, especialmente em situações onde as variáveis não são originalmente gaussianas.
Se você tiver dúvidas sobre o uso do Box-Cox ou a necessidade de outro tipo de transformação, é só avisar!

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture

# Criar transformações
transformacoes = ColumnTransformer(transformers=[
   ('num', Pipeline(steps=[
       #('boxcox', PowerTransformer(method='box-cox')),  # Transformação Box-Cox para aproximar normalidade
       ('scaler', StandardScaler())  # Normalização
   ]), num_cols),
   ('cat', OneHotEncoder(), cat_cols)
])
# Criar pipeline com pré-processamento e GMM
pipeline = Pipeline(steps=[
   ('pre_processamento', transformacoes),
   ('gmm', GaussianMixture(n_components=3, random_state=42))
])
# Ajustar modelo
pipeline.fit(sample_1)
# Prever clusters
clusters = pipeline['gmm'].predict(pipeline['pre_processamento'].transform(sample_1))
print("Clusters:", clusters)

## **Conclusão:**
Após a escolha de 4 grupos através da distãncia intra clusters de elbow,clusterizamos com o algoritmo kmeans e abordagem de distancia euclidiana. Os grupos apresentaram distinção entre quem deu churn ou não, no mes target e outros. Outro  insight relevante foi a distinção de audição entre os grupos, e uma relação observada entre churn e a propria audição dos grupos.